In [3]:
has_pericardial_effusion.head()
Out[3]:

    survival  observed     age  pericardialeffusion  name
11      52.0       1.0  62.000                  1.0  name
15      24.0       1.0  55.000                  1.0  name
16       0.5       0.0  69.000                  1.0  name
17       0.5       0.0  62.529                  1.0  name
19       1.0       0.0  66.000                  1.0  name


In [4]:
none_pericardial_effusion.head()
Out[4]:

   survival  observed   age  pericardialeffusion  name
0      11.0       1.0  71.0                  0.0  name
1      19.0       1.0  72.0                  0.0  name
2      16.0       1.0  55.0                  0.0  name
3      57.0       1.0  60.0                  0.0  name
4      19.0       0.0  57.0                  0.0  name

In [ ]:
#packages are lifeline

In [ ]:
# Instantiate Kaplan Meier object for patients with and without pericardial effusion
kmf_has_pe = KaplanMeierFitter()
kmf_no_pe = KaplanMeierFitter()

# Fit Kaplan Meier estimators to each DataFrame
kmf_has_pe.fit(durations=has_pericardial_effusion['survival'], 
               event_observed=has_pericardial_effusion['observed'])
kmf_no_pe.fit(durations=none_pericardial_effusion['survival'], 
              event_observed=none_pericardial_effusion['observed'])

# Print out the median survival duration of each group
print("The median survival duration (months) of patients with pericardial effusion: ", kmf_has_pe.median_survival_time_)
print("The median survival duration (months) of patients without pericardial effusion: ", kmf_no_pe.median_survival_time_)

In [ ]:
The median survival duration (months) of patients with pericardial effusion:  27.0
The median survival duration (months) of patients without pericardial effusion:  31.0